In [115]:
!pip install sqlalchemy psycopg2 pandas

  Using cached psycopg2-2.9.10.tar.gz (385 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached python_dateutil-2.9.0.post0-py2.py3-none-any.whl.metadata (8.4 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached six-1.17.0-py2.py3-none-any.whl.metadata (1.7 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 30.7 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 51.6 MB/s eta 0:00:00
Using cached python_dateutil-2.9.0.post0-py2.py3-none-any.whl (229 kB)
Using cached pytz-2025.2-py2.py3-none-any.whl (509 kB)
Using cached six-1.17.0-py2.py3-none-any.whl (11 kB)
Using cached tzdata-2025.2-py2.py3-none-any.whl (347 kB)
  Created wheel for psycopg2: filename=psycopg2-2.9.10-cp311-cp311-macosx_14_0_arm64.whl size=144298 sha256=7d5221552c3eb000226595fccf5dcd

In [116]:
from sqlalchemy import create_engine, MetaData
import pandas as pd
import json
from sqlalchemy import create_engine, text

user = "padel_user"
password = "portopadelleague"
host = "35.205.246.86"  # your GCE VM public IP
port = "5432"
database = "padel_league"

# PostgreSQL destination
pg_engine = create_engine(f"postgresql://{user}:{password}@{host}:{port}/{database}")


In [117]:
def get_all_tables():
    """
    Retrieves a list of all table names in the connected PostgreSQL database.

    Returns:
        list: A list of table names (str).
    """
    with pg_engine.connect() as connection:
        result = connection.execute(text("""
            SELECT table_name 
            FROM information_schema.tables 
            WHERE table_schema = 'public' AND table_type = 'BASE TABLE'
        """))
        tables = [row[0] for row in result.fetchall()]
        return tables

In [118]:
def get_columns_and_types(table_name: str):
    """
    Retrieves all column names and their data types from a specific table.

    Args:
        table_name (str): The name of the table.

    Returns:
        list of tuples: Each tuple contains (column_name, data_type).
    """
    with pg_engine.connect() as connection:
        result = connection.execute(text("""
            SELECT column_name, data_type
            FROM information_schema.columns
            WHERE table_name = :table
        """), {"table": table_name})
        
        columns = [(row[0], row[1]) for row in result.fetchall()]
        return columns

In [119]:
def add_column_to_table(table_name: str, column_name: str, column_type: str):
    """
    Adds a new column to an existing PostgreSQL table.

    Args:
        table_name (str): The name of the table to modify.
        column_name (str): The name of the new column.
        column_type (str): The SQL type of the new column (e.g., 'TEXT', 'INTEGER', 'BOOLEAN', etc.).
    """
    with pg_engine.begin() as connection:
        # Check if column already exists
        check_query = text(f"""
            SELECT column_name 
            FROM information_schema.columns 
            WHERE table_name = :table AND column_name = :column
        """)
        result = connection.execute(check_query, {"table": table_name, "column": column_name}).fetchone()

        if result:
            print(f"⚠️ Column '{column_name}' already exists in table '{table_name}'.")
        else:
            alter_query = text(f'ALTER TABLE "{table_name}" ADD COLUMN "{column_name}" {column_type}')
            connection.execute(alter_query)
            print(f"✅ Added column '{column_name}' of type '{column_type}' to table '{table_name}'.")


In [120]:
def fill_column_sequentially(table_name: str, column_name: str):
    """
    Fills a specified column in a table with sequential integer values starting from 1.

    Args:
        table_name (str): The name of the table.
        column_name (str): The name of the column to fill.
    """
    with pg_engine.begin() as connection:
        # PostgreSQL-specific SQL using CTE to generate row numbers
        update_query = text(f"""
            WITH numbered AS (
                SELECT ctid, ROW_NUMBER() OVER () as seq
                FROM "{table_name}"
            )
            UPDATE "{table_name}" AS t
            SET "{column_name}" = n.seq
            FROM numbered n
            WHERE t.ctid = n.ctid
        """)
        connection.execute(update_query)
        print(f"✅ Filled column '{column_name}' in table '{table_name}' with sequential values.")


In [121]:
def fix_missing_sequences():
    """
    Ensures all tables with an 'id' column have an associated sequence
    and sets the default value of the 'id' column to use the sequence.
    """
    tables = get_all_tables()
    with pg_engine.begin() as connection:
        for table in tables:
            # Check if the table has an 'id' column
            id_column_check = connection.execute(text(f"""
                SELECT column_name, data_type 
                FROM information_schema.columns 
                WHERE table_name = :table AND column_name = 'id'
            """), {"table": table}).fetchone()
            
            if not id_column_check:
                continue

            col_name, data_type = id_column_check
            if data_type not in ['integer', 'bigint']:
                continue  # only process integer/bigint ids

            sequence_name = f"{table}_id_seq"

            print(f"Fixing sequence for: {table}")

            # Create the sequence if it doesn't exist
            connection.execute(text(f"""
                CREATE SEQUENCE IF NOT EXISTS {sequence_name}
            """))

            # Set the default value of the id column to use the sequence
            connection.execute(text(f"""
                ALTER TABLE {table} ALTER COLUMN id SET DEFAULT nextval('{sequence_name}')
            """))

            # Set sequence to current MAX(id), or fallback to 1
            connection.execute(text(f"""
                SELECT setval('{sequence_name}', GREATEST((SELECT COALESCE(MAX(id), 0) FROM {table}), 1))
            """))

            print(f"✅ Sequence set for {table}")
        return True

In [122]:
def get_all_elements_from_table(table_name, order_by="id", descending=True):
    """
    Retrieves all rows from the specified table in the PostgreSQL database,
    optionally ordered by a column.

    Args:
        table_name (str): The name of the table to retrieve data from.
        order_by (str): Column name to order by (default is 'id').
        descending (bool): If True, order descending (newest first).

    Returns:
        list: A list of dictionaries representing rows in the table.
    """
    order_clause = f"ORDER BY {order_by} {'DESC' if descending else 'ASC'}"
    with pg_engine.connect() as connection:
        result = connection.execute(text(f"SELECT * FROM {table_name} {order_clause}"))
        rows = result.mappings().all()
        return rows

In [123]:
def delete_elements_with_condition(table_name, column, value):
    """
    Deletes rows from a specified table where a column matches a given value.

    Args:
        table_name (str): The name of the table.
        column (str): The column name to apply the condition.
        value (Any): The value to match for deletion.

    Returns:
        int: Number of rows deleted.
    """
    with pg_engine.begin() as connection:
        result = connection.execute(
            text(f"DELETE FROM {table_name} WHERE {column} = :val"),
            {"val": value}
        )
        return result.rowcount

In [124]:
get_all_tables()

['product_attribute_values',
 'product_attributes',
 'product_images',
 'registrations',
 'players_in_match',
 'product_attribute_values_in_product',
 'product_attributes_in_product',
 'players_in_division',
 'imageables',
 'alembic_version',
 'sponsor_clicks',
 'divisions',
 'images',
 'sponsors',
 'players',
 'news',
 'editions',
 'leagues',
 'matches',
 'order_lines',
 'orders',
 'users',
 'products',
 'backend_app']

In [111]:
def delete_null_match_rows(table_name, player_id):
    """
    Deletes a row for a player where match_id and team are null.

    Args:
        table_name (str): The table name to delete from.
        player_id (int): The player ID to match.

    Returns:
        int: Number of rows deleted.
    """
    with pg_engine.connect() as connection:
        result = connection.execute(
            text(f"""
                DELETE FROM {table_name}
                WHERE player_id = :player_id AND match_id IS NULL AND team IS NULL
            """),
            {"player_id": player_id}
        )
        return result.rowcount
    
#delete_null_match_rows("players_in_match", 26)

In [112]:
#delete_elements_with_condition("players_in_match",'id',6073)

In [125]:
get_all_elements_from_table('images')

[{'id': 263, 'filename': None, 'imageable_id': None, 'object_key': 'images/Backend_App/20250417193842_app_image.png', 'content_type': None, 'size_bytes': None, 'is_public': True},
 {'id': 262, 'filename': None, 'imageable_id': None, 'object_key': 'images/Backend_App/20250421164735_app_image.png', 'content_type': None, 'size_bytes': None, 'is_public': True},
 {'id': 261, 'filename': None, 'imageable_id': None, 'object_key': 'images/Backend_App/20250421164657_app_image.png', 'content_type': None, 'size_bytes': None, 'is_public': True},
 {'id': 260, 'filename': None, 'imageable_id': None, 'object_key': 'images/Backend_App/20250423172152_app_image.png', 'content_type': None, 'size_bytes': None, 'is_public': True},
 {'id': 259, 'filename': None, 'imageable_id': None, 'object_key': 'images/Backend_App/20250417193859_app_image.png', 'content_type': None, 'size_bytes': None, 'is_public': True},
 {'id': 258, 'filename': None, 'imageable_id': None, 'object_key': 'images/Backend_App/2025041719394

In [114]:
get_all_elements_from_table('images')

[]

In [ ]:
#add_column_to_table(table_name='news', column_name='latest', column_type='boolean')

✅ Added column 'latest' of type 'boolean' to table 'news'.


In [ ]:
#fill_column_sequentially(table_name='product_attributes_in_product', column_name='id')

✅ Filled column 'id' in table 'product_attributes_in_product' with sequential values.


In [ ]:
#get_columns_and_types('players_in_match')

[('player_id', 'bigint'),
 ('match_id', 'bigint'),
 ('team', 'text'),
 ('id', 'bigint')]

In [ ]:
#fix_missing_sequences()

Fixing sequence for: product_attribute_values
✅ Sequence set for product_attribute_values
Fixing sequence for: product_attributes
✅ Sequence set for product_attributes
Fixing sequence for: product_images
✅ Sequence set for product_images
Fixing sequence for: backend_app
✅ Sequence set for backend_app
Fixing sequence for: players_in_match
✅ Sequence set for players_in_match
Fixing sequence for: product_attribute_values_in_product
✅ Sequence set for product_attribute_values_in_product
Fixing sequence for: product_attributes_in_product
✅ Sequence set for product_attributes_in_product
Fixing sequence for: players_in_division
✅ Sequence set for players_in_division
Fixing sequence for: images
✅ Sequence set for images
Fixing sequence for: divisions
✅ Sequence set for divisions
Fixing sequence for: editions
✅ Sequence set for editions
Fixing sequence for: leagues
✅ Sequence set for leagues
Fixing sequence for: matches
✅ Sequence set for matches
Fixing sequence for: order_lines
✅ Sequence set 

True

In [17]:
def replace_substring_in_column(table_name, column_name, old_substring, new_substring):
    """
    Replaces a substring in all values of a column in the specified table.

    Args:
        table_name (str): The name of the table to update.
        column_name (str): The name of the column to update.
        old_substring (str): The substring to be replaced.
        new_substring (str): The substring to replace with.

    Returns:
        int: The number of rows updated.
    """
    query = text(f"""
        UPDATE {table_name}
        SET {column_name} = REPLACE({column_name}, :old_substring, :new_substring)
    """)
    with pg_engine.begin() as connection:
        result = connection.execute(query, {"old_substring": old_substring, "new_substring": new_substring})
        return result.rowcount


def add_substring_to_column(table_name, column_name, substring, position="append"):
    """
    Adds a substring to all values of a column in the specified table.

    Args:
        table_name (str): The name of the table to update.
        column_name (str): The name of the column to update.
        substring (str): The substring to add.
        position (str): Whether to 'append' (default) or 'prepend' the substring.

    Returns:
        int: The number of rows updated.
    """
    if position == "prepend":
        expr = f":substring || {column_name}"
    else:  # append
        expr = f"{column_name} || :substring"

    query = text(f"""
        UPDATE {table_name}
        SET {column_name} = {expr}
    """)
    with pg_engine.begin() as connection:
        result = connection.execute(query, {"substring": substring})
        return result.rowcount

In [27]:
add_substring_to_column('news', 'picture_path','images/Player/', 'prepend')

69

In [31]:
replace_substring_in_column('news', 'cover_path', 'images/', 'images/News/')

13

In [74]:
get_all_elements_from_table('news')

[{'id': 13, 'title': 'Porto Padel League - Novo Site e Loja Oficial', 'cover_path': 'images/portopadelleague-novositeoficial_13.jpg', 'author': 'Direção Porto Padel League', 'text': '<h1>Porto Padel League inaugura novo site oficial em domínio próprio: <a href="https://portopadelleague.com" target="_blank">portopadelleague.com</a></h1>    <p>É com enorme entusiasmo e um profundo sentido de orgulho que anunciamos o lançamento do <strong>novo portal digital da Porto Padel League</strong>, agora alojado no domínio <span class="highlight">exclusivo e definitivo: portopadelleague.com</span>. Esta não é apenas uma atualização — é uma <strong>reinvenção completa da nossa presença online</strong>, projetada para refletir a ambição, o profissionalismo e a excelência da nossa comunidade.</p>    <p>Este feito extraordinário é o resultado de uma <strong>colaboração interdepartamental sem precedentes</strong>, que uniu mentes brilhantes e talento técnico de várias áreas:</p>    <div class="section"